In [3]:
import pyvips
import torch, torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset as Dataset
from torch.autograd import Variable
import torch.nn.functional as F

from torchvision import datasets, models
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
plt.switch_backend('agg')

import time, os, copy, datetime, glob
from tqdm import tqdm
from skimage import io
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

format_to_dtype = {
    'uchar': np.uint8,
    'char': np.int8,
    'ushort': np.uint16,
    'short': np.int16,
    'uint': np.uint32,
    'int': np.int32,
    'float': np.float32,
    'double': np.float64,
    'complex': np.complex64,
    'dpcomplex': np.complex128,
}

# vips image to numpy array
def vips2numpy(vi):
    return np.ndarray(buffer=vi.write_to_memory(),
                      dtype=format_to_dtype[vi.format],
                      shape=[vi.height, vi.width, vi.bands])
print(torch.__version__, torchvision.__version__)

1.4.0 0.5.0


In [4]:
NORM_PATH = '/BrainSeg/normalization.npy'
norm = np.load(NORM_PATH,allow_pickle=True).item()
print(norm)

{'mean': array([0.77906426, 0.74919518, 0.77529276]), 'std': array([0.13986633, 0.15931302, 0.17665639])}


In [4]:
trans = transforms.Compose([
#                           transforms.RandomHorizontalFlip(),
#                           # transforms.Resize((32,32)),
#                           transforms.RandomVerticalFlip(),
#                           transforms.RandomRotation(180),
#                           transforms.ColorJitter(brightness=0.1, contrast=0.2,saturation=0.2, hue=0.02),
#                           transforms.RandomAffine(0, translate=(0.05,0.05), scale=(0.9,1.1), shear=10),
                          transforms.ToTensor(),
#                           transforms.Normalize(norm['mean'], norm['std'])
                          ])
# from PIL import Image
# from torch.autograd import Variable as V
# model = torch.load(BASE_DIR + 'new_train/resnet18_3.pkl')
model = torch.load('/BrainSeg/Checkpoints/new_LF/ResNet18_19.pkl')
model = model.cuda()
model = model.eval()

In [5]:
# Specify images to run
BASE_DIR = '/BrainSeg/'
IMG_DIR  = BASE_DIR + 'HE_PNG/'
SAVE_DATA_DIR = BASE_DIR + 'Classify_Results/HE0801/GMBData/'
if not os.path.exists(SAVE_DATA_DIR):
    os.makedirs(SAVE_DATA_DIR)
print(SAVE_DATA_DIR)
SAVE_IMG_DIR = BASE_DIR + 'Classify_Results/HE0801/GMBMasks/'
if not os.path.exists(SAVE_IMG_DIR):
    os.makedirs(SAVE_IMG_DIR)
print(SAVE_IMG_DIR)

filenames = glob.glob(IMG_DIR + '*.png')
filenames = [filename.split('/')[-1] for filename in filenames]
filenames = sorted(filenames)
total_image_num = len(filenames)

idx_image_to_eval = slice(0,30)      # Modify this line to select images to eval
filenames = filenames[idx_image_to_eval]
print('Run for the following %d images out of %d images:\n' % (len(filenames), total_image_num), filenames)

/BrainSeg/Classify_Results/HE0801/GMBData/
/BrainSeg/Classify_Results/HE0801/GMBMasks/
Run for the following 29 images out of 29 images:
 ['NA3777-02_HE.png', 'NA4077-02_HE.png', 'NA4092-02_HE.png', 'NA4107-02_HE.png', 'NA4160-02_HE.png', 'NA4195-02_HE.png', 'NA4256-02_HE.png', 'NA4299-02_HE.png', 'NA4391-02_HE.png', 'NA4463-02_HE.png', 'NA4471-02_HE.png', 'NA4553-02_HE.png', 'NA4626-02_HE.png', 'NA4672-02_HE.png', 'NA4675-02_HE.png', 'NA4691-02_HE.png', 'NA4695-02_HE.png', 'NA4894-02_HE.png', 'NA4907-02_HE.png', 'NA4944-02_HE.png', 'NA4945-02_HE.png', 'NA4967-02_HE.png', 'NA4971-02_HE.png', 'NA4972-02_HE.png', 'NA4992-02_HE.png', 'NA4993-02_HE.png', 'NA5010-02_HE.png', 'NA5015-02_HE.png', 'NA5029-02_HE.png']


In [ ]:
# Start evaluating
t = tqdm(total=len(filenames))
# print(filenames)
for filename in filenames:
  # print(filename)
  # in_img = np.array(Image.open(IMG_DIR + filename)) # Out of RAM
    in_img = pyvips.Image.new_from_file(IMG_DIR + filename)
  
    t.set_description_str("Image " + filename + ' - (%d, %d)' % (in_img.width, in_img.height))
    t.refresh()
    t.write("", end=' ')

    num_cols = int(in_img.width/128)
    num_rows = int(in_img.height/128)
    nums = np.zeros((num_rows, num_cols), dtype='uint8')
#     PRED = np.zeros((num_rows, num_cols, 3), dtype='uint8')

    for i in range(num_rows):
        for j in range(num_cols):
      #print('i, j = {}, {}'.format(i, j))
            w, h = 256, 256
            if i == num_rows - 1: # if at last row
                h = 128
            if j == num_cols - 1: # if at last column
                w = 128
            tile_img = in_img.extract_area(128*j,128*i,w,h) # c, r, w, h
            tile_img = vips2numpy(tile_img)
          #print(tile_img.shape)
            if i == num_rows - 1: # if at last row
                tile_img = np.pad(tile_img, ((0, 128),(0, 0),(0, 0)), 'constant', constant_values=((0, 0),))
        #print('i, j = {}, {}'.format(i, j), tile_img.shape)
            if j == num_cols - 1: # if at last column
                tile_img = np.pad(tile_img, ((0, 0),(0, 128),(0, 0)), 'constant', constant_values=((0, 0),))
        #print('i, j = {}, {}'.format(i, j), tile_img.shape)
            tile_img = Image.fromarray(tile_img, 'RGB')
            tile_img = trans(tile_img)
            tile_img = tile_img.unsqueeze(0)
            tile_img = Variable(tile_img)
            tile_img = tile_img.cuda()
            predict = model(tile_img)

            # a = predict.data[0,0].item()
            # b = predict.data[0,1].item()
#             # c = predict.data[0,2].item()
#             PRED[i, j, 0] = predict.data[0,0].item()
#             PRED[i, j, 1] = predict.data[0,1].item()
#             PRED[i, j, 2] = predict.data[0,2].item()
            # print('a:',a,'b:',b,'c',c)
            # print('predict.data',predict.data)
            _, predict = torch.max(predict.data, 1)
            value = predict.item()
            # print(value)
            #print(type(value)) #<class 'int'>
            nums[i, j] = value
      
    np.save(SAVE_DATA_DIR + filename.split('.')[-2] + '.npy', nums)
#     np.save(SAVE_DATA_DIR + filename.split('.')[-2] + 'value.npy', PRED)
    nums = np.repeat(nums[:, :, np.newaxis], 3, axis=2)
    #print(nums.shape, nums.dtype)

    # nums[:,:,0] = RED, nums[:,:,1] = Green, nums[:,:,2] = Blue
    idx_1 = np.where(nums[:,:,0] == 2)  # Index of label 1 (WM)
    idx_2 = np.where(nums[:,:,0] == 1)  # Index of label 2 (GM)

    # For label 0, leave as black color
    # For label 1, set to cyan color: R0G255B255
    nums[:,:,0].flat[np.ravel_multi_index(idx_1, nums[:,:,0].shape)] = 0
    nums[:,:,1].flat[np.ravel_multi_index(idx_1, nums[:,:,1].shape)] = 255
    nums[:,:,2].flat[np.ravel_multi_index(idx_1, nums[:,:,2].shape)] = 255
    # For label 2, set to yellow color: R255G255B0
    nums[:,:,0].flat[np.ravel_multi_index(idx_2, nums[:,:,0].shape)] = 255
    nums[:,:,1].flat[np.ravel_multi_index(idx_2, nums[:,:,1].shape)] = 255
    nums[:,:,2].flat[np.ravel_multi_index(idx_2, nums[:,:,2].shape)] = 0

    save_img = Image.fromarray(nums, 'RGB')
    save_img.save(SAVE_IMG_DIR + filename.split('.')[-2] + '.png')

    t.update()
t.close()

Image NA3777-02_HE.png - (53784, 44411):   0%|          | 0/29 [00:00<?, ?it/s]

Image NA4077-02_HE.png - (73704, 44001):   3%|▎         | 1/29 [38:20<17:53:43, 2300.85s/it]

Image NA4092-02_HE.png - (63744, 45102):   7%|▋         | 2/29 [1:29:52<19:02:05, 2537.97s/it]